In [1]:
import torch
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from dgl.nn import GraphConv,SAGEConv
import dgl
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
import seaborn as sns
import random
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split
import os
import yaml
import time
import datetime
from modules import ICPKGIDataset
from models import PatchGCN,MultiPatchGCN

In [3]:
'''
データセットを読み込む
データローダーをシャッフルで作成
モデルにデータを入力
1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る
出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする
テストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う
学習における分類精度と特徴グラフによる分類精度を計算し保存する
分類学習における予測と正解を用いて損失を計算しネットワークを更新する
すべてのテストデータに対して行ったのち全epochについて行う
このファイルは1種類の物体について分類を行うファイルとする
'''

'\nデータセットを読み込む\nデータローダーをシャッフルで作成\nモデルにデータを入力\n1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る\n出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする\nテストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う\n学習における分類精度と特徴グラフによる分類精度を計算し保存する\n分類学習における予測と正解を用いて損失を計算しネットワークを更新する\nすべてのテストデータに対して行ったのち全epochについて行う\nこのファイルは1種類の物体について分類を行うファイルとする\n'

In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
object_name = 'airplane'  #car bus airplane
setting_file = "config2.yaml"

#データ読み込み
dataset=ICPKGIDataset(f'../data/ICPKGI/8patch_gray_{object_name}.dgl')

#各クラスから均等に10個ずつテスト用として抜き出しtrainデータセットとtestデータセットを作成
labels=[i.item() for _,i in dataset]
traindataset, testdataset, trainlabels, testlabels=train_test_split(dataset,labels,test_size=0.2,shuffle=True,stratify=labels)

#データローダー作成
traindataloader=GraphDataLoader(traindataset,batch_size=16,shuffle=True,num_workers = 0,pin_memory = True)
testdataloader=GraphDataLoader(testdataset,batch_size=10,shuffle=True,num_workers = 0,pin_memory = True)


In [31]:
for i,(g,d) in enumerate(traindataloader):
    break
print(g,d)

Graph(num_nodes=1024, num_edges=6720,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32)}
      edata_schemes={}) tensor([0, 0, 0, 3, 3, 2, 3, 4, 3, 0, 0, 4, 1, 1, 1, 1])


In [6]:
#設定ファイル読み込み
with open(f'./configs/{setting_file}','r') as f:
    config = yaml.safe_load(f)

#モデルの初期化
model=PatchGCN(1024,[8],5,embedding=True)
model.to(device)

PatchGCN(
  (input_layer): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=1024, out_features=8, bias=False)
    (fc_self): Linear(in_features=1024, out_features=8, bias=True)
  )
  (middle_layers): ModuleList()
  (output_layer): GraphConv(in=8, out=5, normalization=both, activation=None)
  (m): LeakyReLU(negative_slope=0.01)
  (flatt): Flatten(start_dim=1, end_dim=-1)
)

In [26]:
emb_graphs=[]
emb_labels=[]
for i,(batched_graph,labels) in enumerate(traindataloader):
    emb_labels.extend(labels.tolist())
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)

    pred,emb = model(batched_graph,batched_graph.ndata['f'])
    emb_graphs.extend(dgl.unbatch(emb))
    if i == 1:
        break
print(i)
test_emb_graphs=[]
test_emb_labels=[]
#テスト
model.eval()
for tbatched_graph, tlabels in testdataloader:
    test_emb_labels.extend(tlabels.tolist())
    tbatched_graph = tbatched_graph.to(device)
    tlabels = tlabels.to(device)
    tpred,temb = model(tbatched_graph, tbatched_graph.ndata['f'])
    test_emb_graphs.extend(dgl.unbatch(temb))

1


In [27]:
print(len(test_emb_graphs))
stack_test_emb = torch.stack([g.ndata['emb'].to('cpu') for g in test_emb_graphs],dim=0).unsqueeze(1)

559


In [28]:
print(stack_test_emb.shape)

torch.Size([559, 1, 64, 8])


In [14]:
unbatch_emb=dgl.unbatch(emb)
print(unbatch_emb)

[Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), 

In [15]:
print(unbatch_emb[0].ndata['emb'].shape)
print(labels)

torch.Size([64, 8])
tensor([3, 1, 0, 4, 3, 3, 2, 2, 2, 1, 4, 4, 0, 3, 4, 4], device='cuda:1')


In [20]:
print(len(emb_graphs))
print(len(emb_labels))
print(emb_labels)

32
32
[4, 0, 3, 3, 3, 2, 0, 3, 3, 1, 4, 3, 4, 3, 3, 3, 1, 3, 2, 3, 3, 3, 3, 3, 0, 2, 3, 3, 4, 4, 1, 0]


In [9]:
direction_graphs=torch.zeros(5,64,8) #中間層の出力のクラス特徴を保存するリスト
direction_labels=[0]*5 #ラベル数が何枚あるかを示すリスト
for i in emb_labels:
    direction_labels[i]+=1
for emb_graph,emb_label in zip(emb_graphs,emb_labels):
    #中間層の出力をラベル数で割ってクラスインデックスに加算する
    direction_graphs[emb_label]+=((emb_graph.ndata['emb'])/direction_labels[emb_label]).to('cpu')


In [11]:
print(direction_graphs.shape)
print(direction_labels)

torch.Size([5, 64, 8])
[3, 6, 1, 8, 14]


In [30]:

i1=torch.randn(5,4,2)
i2=torch.randn(10,1,4,2)
cos=nn.CosineSimilarity(-1)
out=cos(i1,i2)
print(out.shape)
#print(torch.sum(out,2))

torch.Size([10, 5, 4])


In [107]:
print(out.reshape(2,5,2,2))

tensor([[[[-0.6558, -0.7974],
          [-0.0419,  0.1725]],

         [[ 0.5432, -0.0453],
          [-0.8530,  0.2272]],

         [[-0.8122,  0.0120],
          [ 0.9431,  0.6192]],

         [[ 0.2029, -0.6699],
          [-0.4704,  0.9742]],

         [[ 0.9992,  0.9900],
          [ 0.9499, -0.9605]]],


        [[[ 0.1627, -0.9442],
          [-0.0737,  0.6008]],

         [[-0.0224,  0.8065],
          [ 0.9075,  0.6445]],

         [[ 0.3859,  0.8390],
          [-0.9752,  0.9082]],

         [[-0.6861,  0.2468],
          [ 0.5691,  0.9706]],

         [[-0.8302,  0.6658],
          [-0.9796, -0.9822]]]])


In [37]:
out1=cos(direction_graphs,stack_test_emb)
print(out1.shape)
emb_pred=torch.sum(out1,dim=-1)
print(emb_pred[0])
print((emb_pred.argmax(1)==torch.tensor(test_emb_labels)).sum().item())

torch.Size([559, 5, 64])
tensor([63.1343, 63.3886, 61.6226, 63.4640, 63.4585],
       grad_fn=<SelectBackward0>)
144


# <span style="color: red;">以下物体ごとの分類を行うコード</span>

In [8]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
object_name = 'all'  #car bus airplane
setting_file = "config_all.yaml"

#データ読み込み
dataset=ICPKGIDataset(f'../data/ICPKGI/8patch_gray_{object_name}.dgl')

#各クラスから均等に2割りずつテスト用として抜き出しtrainデータセットとtestデータセットを作成
labels=[i.item() for _,i in dataset]
traindataset, testdataset, trainlabels, testlabels=train_test_split(dataset,labels,test_size=0.2,shuffle=True,stratify=labels)

#データローダー作成
traindataloader=GraphDataLoader(traindataset,batch_size=16,shuffle=True,num_workers = 0,pin_memory = True)
testdataloader=GraphDataLoader(testdataset,batch_size=10,shuffle=True,num_workers = 0,pin_memory = True)

In [4]:
for i,(g,d) in enumerate(traindataloader):
    break
print(g,d)

Graph(num_nodes=1024, num_edges=6720,
      ndata_schemes={'d': Scheme(shape=(1,), dtype=torch.int64), 'f': Scheme(shape=(1, 32, 32), dtype=torch.float32)}
      edata_schemes={}) tensor([2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 0])


In [3]:
#モデルの初期化
model=MultiPatchGCN(1024,[8],object_output_size=5,direction_output_size=3)
model.to(device)

MultiPatchGCN(
  (input_layer): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=1024, out_features=8, bias=False)
    (fc_self): Linear(in_features=1024, out_features=8, bias=True)
  )
  (middle_layers): ModuleList()
  (object_output_layer): GraphConv(in=8, out=5, normalization=both, activation=None)
  (direction_output_layer): GraphConv(in=8, out=3, normalization=both, activation=None)
  (m): LeakyReLU(negative_slope=0.01)
  (flatt): Flatten(start_dim=1, end_dim=-1)
)

In [10]:
traindataset[0][0].ndata['d'][0].item()

4

In [9]:
direction_labels=torch.zeros(3,5) #トレーニングデータにおける各物体各方向ごとのデータ数
for g,i in traindataset:
    direction_labels[i][g.ndata['d'][0].item()]+=1
print(direction_labels)

tensor([[398., 486., 398., 549., 401.],
        [465., 471., 406., 541., 457.],
        [462., 599., 394., 420., 433.]])


In [10]:
emb_graphs=[]
emb_labels=[]
for i,(batched_graph,labels) in enumerate(traindataloader):
    emb_labels.extend(labels.tolist())
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)

    obj_pred,dir_pred,emb = model(batched_graph,batched_graph.ndata['f'])
    emb_graphs.extend(dgl.unbatch(emb))
    if i == 1:
        break
print(i)
test_emb_graphs=[]
test_emb_labels=[]
#テスト
model.eval()
for tbatched_graph, tlabels in testdataloader:
    test_emb_labels.extend(tlabels.tolist())
    tbatched_graph = tbatched_graph.to(device)
    tlabels = tlabels.to(device)
    tobj_pred,tdir_pred,temb = model(tbatched_graph, tbatched_graph.ndata['f'])
    test_emb_graphs.extend(dgl.unbatch(temb))

1


In [11]:
print(emb_graphs[0])


Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'d': Scheme(shape=(1,), dtype=torch.int64), 'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'oc': Scheme(shape=(5,), dtype=torch.float32), 'dc': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})


In [16]:
direction_graphs=torch.zeros(3,5,64,8) #中間層の出力のクラス特徴を保存するリスト
for emb_graph,emb_label in zip(emb_graphs,emb_labels):
    #中間層の出力をラベル数で割ってクラスインデックスに加算する
    dir_label=emb_graph.ndata['d'][0].item()
    direction_graphs[emb_label][dir_label]+=((emb_graph.ndata['emb'])/direction_labels[emb_label][dir_label]).to('cpu')


In [17]:
print(direction_graphs.shape)
print(direction_labels)

torch.Size([3, 5, 64, 8])
tensor([[398., 486., 398., 549., 401.],
        [465., 471., 406., 541., 457.],
        [462., 599., 394., 420., 433.]])


In [18]:
#テストデータの出力の埋め込み表現のみをスタックする
stack_test_emb = torch.stack([g.ndata['emb'].to('cpu') for g in test_emb_graphs],dim=0).unsqueeze(1)
print(stack_test_emb.shape)

torch.Size([1721, 1, 64, 8])


In [21]:
cos=nn.CosineSimilarity(-1)
out=cos(direction_graphs[0],stack_test_emb)
print(out.shape)

torch.Size([1721, 5, 64])


In [41]:
emb_pred=torch.stack([cos(stack_test_emb,direction_graph) for direction_graph in direction_graphs],dim=1)
print(emb_pred.shape)
emb_pred=torch.sum(emb_pred,dim=-1)
print(emb_pred.shape)
print(emb_pred[0])

torch.Size([1721, 3, 5, 64])
torch.Size([1721, 3, 5])
tensor([[62.5373, 62.3928, 62.4814, 62.3599, 62.1956],
        [ 0.0000,  0.0000, 61.3671, 62.0113,  0.0000],
        [61.3048, 62.2262,  0.0000, 61.6954, 62.4865]],
       grad_fn=<SelectBackward0>)


In [39]:
a=torch.arange(15).reshape(3,5)
a[1,2]=14
a[2,4]=12
print(a)
print(a.shape)

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6, 14,  8,  9],
        [10, 11, 12, 13, 12]])
torch.Size([3, 5])


In [46]:
print(torch.max(torch.max(a,dim=1).values,dim=0))
last_emb_pred=torch.max(torch.max(emb_pred,dim=2).values,dim=1).indices
print(last_emb_pred[:10])

torch.return_types.max(
values=tensor(14),
indices=tensor(1))
tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 0])


In [47]:
b=torch.zeros(3)
for i in last_emb_pred:
    b[i]+=1
print(b)

tensor([1507.,   49.,  165.])
